In [ ]:
import random, math

from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
import torch


In [ ]:
#model = AutoModelForCausalLM.from_pretrained('roneneldan/TinyStories-33M')
#model = AutoModelForCausalLM.from_pretrained('roneneldan/TinyStories-Instruct-33M')
model = AutoModelForCausalLM.from_pretrained("blueapple8259/TinyStories-Alpaca")

tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-125M")

In [ ]:
model.config.max_position_embeddings

In [ ]:
%%time

def iter_auto_complete(
    prompt: str = "Once upon a time there was",
    temperature: float = 1.,
    max_length: int = 100,
    num_beams: int = 1,
):
    #prompt += tokenizer.
    print(prompt)
    input_ids = tokenizer.encode(prompt, return_tensors="pt")

    try:
        for i in range(max_length):
            next_logits = model(input_ids)["logits"]
            print(input_ids.shape, next_logits.shape)
            #print(input_ids)
            #print(next_logits.argmax(dim=-1))
            #print(tokenizer.decode(input_ids[0]))
            #print(tokenizer.decode(next_logits.argmax(dim=-1)[0]))

            next_ids = next_logits.argmax(dim=-1)
            print(input_ids[0, 1:])
            print(next_ids[0, :-1])
            num_correct = (next_logits.argmax(dim=-1)[0, :-1] == input_ids[0, 1:]).sum() #/ input_ids.shape[-1]
            print("CORR", num_correct)
            
            next_logits = next_logits[:, -1]
            next_id = next_logits.argmax(dim=-1)#.unsqueeze(-1)
            yield tokenizer.decode(next_id)
            input_ids = torch.concat([input_ids, next_id.unsqueeze(0)], dim=-1)
        
    #    output_ids = model.generate(input_ids, max_length=1, num_beams=num_beams, temperature=temperature, do_sample=temperature != 1)
    #    output_ids = output_ids[:, input_ids.shape[-1]:]
    #    input_ids = torch.concat([input_ids, output_ids], dim=-1)
    #    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    #    yield output_text
        
    except KeyboardInterrupt:
        pass
        

for token in iter_auto_complete(
    #"When Lucy was 28 years old,"
    #"The blood-splattered cyberpunk said:",
    #"After ripping off her head",
    #"One day I went to the park with my big new car",
    #"The scray monster said, \"I will ear your brains",
    #"It's important to take serious risk", 
    #"Never think before acting",
    #"Summary: Two astoroids collide\nFeatures: explosion, devastation\nSentence: This tore apart the whole galaxy\nWords: cyber, space\nStory: In a future life,",
    #"Summary: A story that teaches children to always follow their leader\nFeatures: a crazy bear\nSentence: Always follow your leader!\nWords: nice, vanilla, nuclear\nStory: ",
    #"A story about the", 
    #"What is a collateral biscuit?",
    #"A consecutive sequence: 1, 2, 3, 4",
):
    print(token, end="")

In [ ]:
%%time

def iter_auto_complete_prev(
    prompt: str = "Once upon a time there was",
    temperature: float = 1.,
    max_length: int = 100,
    num_beams: int = 1,
):
    #prompt += tokenizer.
    print(prompt)

    past_key_values = None
    next_id = tokenizer.encode(prompt, return_tensors="pt")
    try:
        for i in range(max_length):
            next_logits, past_key_values = model(next_id, use_cache=True, past_key_values=past_key_values).to_tuple()
            next_logits = next_logits[:, -1:]
            next_id = next_logits.argmax(dim=-1)#.unsqueeze(-1)
            yield tokenizer.decode(next_id.item())
            # input_ids = torch.concat([input_ids, next_id.unsqueeze(0)], dim=-1)
        
    except KeyboardInterrupt:
        pass
        

for token in iter_auto_complete_prev():
    print(token, end="")

In [ ]:
%%time

def iter_auto_complete_sample(
    prompt: str = "1, 2, 3, 4,",  #"Once upon a time there was",
    max_length: int = 100,
    top_k: int = 5,
):
    #prompt += tokenizer.
    print(prompt)

    past_key_values = None
    next_id = tokenizer.encode(prompt, return_tensors="pt")
    try:
        for i in range(max_length):
            next_logits, past_key_values = model(next_id, use_cache=True, past_key_values=past_key_values).to_tuple()
            next_logits = next_logits[:, -1:]

            sorted_logit_ids = torch.argsort(next_logits, dim=-1, descending=True)

            idx = int(math.pow(random.random() * random.random(), 5) * top_k)
            #print(sorted_logit_ids)
            next_id = sorted_logit_ids[:, :, idx]
            #next_id = next_logits.argmax(dim=-1)
            
            yield tokenizer.decode(next_id.item(), skip_special_tokens=True)
            # input_ids = torch.concat([input_ids, next_id.unsqueeze(0)], dim=-1)
        
    except KeyboardInterrupt:
        pass
        

for token in iter_auto_complete_sample():
    print(token, end="")

In [ ]:
print(f"  /\\  \n /  \\ \n/____\\\n")